In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import sys
from pathlib import Path
import os
PROJECT_ROOT = Path.cwd().parents[0]
SRC_PATH = PROJECT_ROOT / "src"
sys.path.append(str(SRC_PATH))

from MicroNAV.adapters.outbound.db.sqlite_db import Db_datos


from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[0]
DB_PATH = PROJECT_ROOT / "data" / "microvap.db"

db_data = Db_datos(DB_PATH)


/home/versekert/Documentos/Ucts/MicroNAV/data/microvap.db


In [43]:
df = pd.read_excel(DATA_RAW / "microvap_metadata_complete_15012026.xlsx")
df.columns = df.columns.str.strip()
df.head()

,specimen,copies_ul_dna,patient_id,timepoint,timepoint_character,tissue_type,admission_date_cus,admission_date_icu,days_stay_icu,pneumonia,...,Otu16160,Otu16185,Otu16895,Otu17366,Otu17509,Otu17573,Otu17638,Otu18078,Otu18302,Otu18444
0,006_1_BAL,3459.212646,6,uno,baseline,bal,2020-01-28,2020-01-28,5,no_vap,...,0,0.0,0,0.0,0,0,0.0,0,0,0
1,006_1_NASAL,NaN,6,uno,baseline,nasal,2020-01-28,2020-01-28,5,no_vap,...,0,0.0,0,0.0,0,0,0.0,0,0,0
2,006_1_TS,14486.053060,6,uno,baseline,ts,2020-01-28,2020-01-28,5,no_vap,...,0,0.0,0,0.0,0,0,0.0,0,0,0
3,011_1_BAL,804.843343,11,uno,baseline,bal,2020-02-05,2020-02-05,12,no_vap,...,0,0.0,0,0.0,0,0,0.0,0,0,0
4,011_1_NASAL,NaN,11,uno,baseline,nasal,2020-02-05,2020-02-05,12,no_vap,...,0,0.0,0,0.0,0,0,0.0,0,0,0


## Update table Patient

In [44]:
variables_list= ['patient_id',
                'gender',
                'height',
                'weight',
                'alcoholism',
                'anemia',
                'stroke',
                'asthma',
                'cancer',
                'diabetes',
                'coronary_heart_disease',
                'mental_illness',
                'chronic_kidney_disease',
                'copd',
                'heart_failure',
                'hemodialysis',
                'hypertension',
                'ppi_chronic',
                'obesity',
                'sahos',
                'smoking',
                'statin',
                'no_background']

df_patient = df[variables_list]
df_patient.head()

,patient_id,gender,height,weight,alcoholism,anemia,stroke,asthma,cancer,diabetes,...,copd,heart_failure,hemodialysis,hypertension,ppi_chronic,obesity,sahos,smoking,statin,no_background
0,6,1,163.0,70.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,6,1,163.0,70.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,6,1,163.0,70.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,11,1,167.0,60.0,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,11,1,167.0,60.0,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0


In [45]:
df_grouped = (
    df_patient
    .groupby('patient_id', as_index=False)
    .agg({
        'gender': 'first',
        'height': 'max',
        'weight': 'max',
        'alcoholism': 'max',
        'anemia': 'max',
        'stroke': 'max',
        'asthma': 'max',
        'cancer': 'max',
        'diabetes': 'max',
        'coronary_heart_disease': 'max',
        'mental_illness': 'max',
        'chronic_kidney_disease': 'max',
        'copd': 'max',
        'heart_failure': 'max',
        'hemodialysis': 'max',
        'hypertension': 'max',
        'ppi_chronic': 'max',
        'obesity': 'max',
        'sahos': 'max',
        'smoking': 'max',
        'statin': 'max',
        'no_background': 'max'
    })
)

In [46]:
df_grouped[df_grouped.duplicated(subset='patient_id', keep=False)] \
    .sort_values('patient_id')

,patient_id,gender,height,weight,alcoholism,anemia,stroke,asthma,cancer,diabetes,...,copd,heart_failure,hemodialysis,hypertension,ppi_chronic,obesity,sahos,smoking,statin,no_background


In [51]:
n = db_data.load_df(df_grouped, "patient", clear_table=True)
print(f"Insertados: {n}")

Insertados: 140


## Update table Admission